Feast - Feature Store
Deimer De Jesús Ballesteros
Simón Dávila

In [2]:
%rm -r taller_2

In [3]:
! feast init taller_2


Creating a new Feast repository in /work/taller_2.



In [4]:
%cd taller_2/feature_repo

/work/taller_2/feature_repo


In [5]:
import pandas as pd
from datetime import datetime, timedelta
import sqlite3
 
from feast import FeatureStore

raw_df=pd.read_parquet("data/driver_stats.parquet")
raw_df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2023-05-07 23:00:00+00:00,1005,0.705529,0.756371,110,2023-05-22 23:31:07.626
1,2023-05-08 00:00:00+00:00,1005,0.012319,0.648218,144,2023-05-22 23:31:07.626
2,2023-05-08 01:00:00+00:00,1005,0.445981,0.410704,381,2023-05-22 23:31:07.626
3,2023-05-08 02:00:00+00:00,1005,0.796583,0.460959,875,2023-05-22 23:31:07.626
4,2023-05-08 03:00:00+00:00,1005,0.555176,0.595937,166,2023-05-22 23:31:07.626


In [6]:
! feast apply

/usr/local/lib/python3.10/site-packages/feast/feature_store.py:561: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
/usr/local/lib/python3.10/site-packages/feast/infra/offline_stores/file_source.py:161: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  schema = ParquetDataset(path).schema
Created entity driver
Created feature view driver_hourly_stats
Created feature view driver_hourly_stats_fresh
Created on demand feature view transformed_conv_rate
Created on demand feature view transformed_conv_rate_fresh
Created feature service driver_activity_v2
Created feature service driver_activity_v3
Created feature servic

In [7]:
dict_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003],
        "label_driver_reported_satisfaction": [1, 5, 3], 
        "event_timestamp": [
            datetime.now() - timedelta(minutes=11),
            datetime.now() - timedelta(minutes=36),
            datetime.now() - timedelta(minutes=73),
        ],
    }
)
 
store = FeatureStore(repo_path=".")
 
training_data = store.get_historical_features(
    entity_df=dict_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
).to_df()

In [8]:
! feast materialize-incremental {datetime.now().isoformat()}

Materializing 2 feature views to 2023-05-22 18:37:18-05:00 into the sqlite online store.

driver_hourly_stats from 2023-05-22 04:37:44-05:00 to 2023-05-22 18:37:18-05:00:
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 99.54it/s]
driver_hourly_stats_fresh from 2023-05-22 04:37:45-05:00 to 2023-05-22 18:37:18-05:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 201.11it/s]


In [10]:
con_online = sqlite3.connect("data/online_store.db")
print(
    pd.read_sql_query(
        "SELECT * FROM taller_2_driver_hourly_stats", con_online))
con_online.close()

                                           entity_key     feature_name  \
0   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
1   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
2   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
3   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
4   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
5   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
6   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
7   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
8   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
9   b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...        conv_rate   
10  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...         acc_rate   
11  b'\x02\x00\x00\x00driver_id\x04\x00\x00\x00\x0...  avg_daily_trips   
12  b'\x02\x00\x00\x00driver_id\x04\x0